In [5]:
import asyncio
import json
import os

import redis.asyncio as redis
from dotenv import load_dotenv


async def subscribe_and_print():
    # Initialize Redis connection
    load_dotenv()
    r = redis.Redis(
        host=os.getenv("REDIS_HOST"), port=os.getenv("REDIS_PORT"), db=0
    ).pubsub()

    # Subscribe to channels
    await r.subscribe("gpu_stats", "cpu_stats", "ssh_stats")

    # Print 10 messages total across all channels
    count = 0
    try:
        while count < 10:
            message = await r.get_message()
            if message and message["type"] == "message":
                data = json.loads(message["data"])
                print(f"Channel: {message['channel']}")
                print(f"Data: {json.dumps(data, indent=2)}")
                print("-" * 50)
                count += 1
    finally:
        await r.unsubscribe()
        await r.aclose()


# Run the async function
await subscribe_and_print()


Channel: b'gpu_stats'
Data: [
  {
    "gpu_id": 0,
    "memory_total": 0,
    "memory_used": 0,
    "memory_free": 0,
    "gpu_utilization": 0,
    "memory_utilization": 0,
    "temperature": 0,
    "timestamp": "2024-11-21T18:17:42.120208"
  }
]
--------------------------------------------------
Channel: b'cpu_stats'
Data: {
  "cpu_utilization": 10.7,
  "memory_total": 8219537408,
  "memory_used": 2174181376,
  "memory_free": 5836181504,
  "memory_percent": 29.0,
  "disk_total": 62671097856,
  "disk_used": 13032919040,
  "disk_free": 46421463040,
  "disk_percent": 21.9,
  "timestamp": "2024-11-21T18:17:43.127090"
}
--------------------------------------------------
Channel: b'ssh_stats'
Data: []
--------------------------------------------------
Channel: b'gpu_stats'
Data: [
  {
    "gpu_id": 0,
    "memory_total": 0,
    "memory_used": 0,
    "memory_free": 0,
    "gpu_utilization": 0,
    "memory_utilization": 0,
    "temperature": 0,
    "timestamp": "2024-11-21T18:17:53.143431"
  

CancelledError: 

In [2]:
import nest_asyncio  # Add this import at the top

nest_asyncio.apply()  # Apply patch to allow nested event loops


import asyncio
import json

import websockets


async def test_websocket():
    async with websockets.connect("ws://localhost:8000/ws/stats/gpu") as websocket:
        while True:
            data = await websocket.recv()
            print(f"Received GPU stats: {json.loads(data)}")


asyncio.run(test_websocket())

Received GPU stats: [{'gpu_id': 0, 'memory_total': 0, 'memory_used': 0, 'memory_free': 0, 'gpu_utilization': 0, 'memory_utilization': 0, 'temperature': 0, 'timestamp': '2024-11-21T18:52:23.690746'}]
Received GPU stats: [{'gpu_id': 0, 'memory_total': 0, 'memory_used': 0, 'memory_free': 0, 'gpu_utilization': 0, 'memory_utilization': 0, 'temperature': 0, 'timestamp': '2024-11-21T18:52:34.708653'}]
Received GPU stats: [{'gpu_id': 0, 'memory_total': 0, 'memory_used': 0, 'memory_free': 0, 'gpu_utilization': 0, 'memory_utilization': 0, 'temperature': 0, 'timestamp': '2024-11-21T18:52:45.724268'}]
Received GPU stats: [{'gpu_id': 0, 'memory_total': 0, 'memory_used': 0, 'memory_free': 0, 'gpu_utilization': 0, 'memory_utilization': 0, 'temperature': 0, 'timestamp': '2024-11-21T18:52:56.759927'}]
Received GPU stats: [{'gpu_id': 0, 'memory_total': 0, 'memory_used': 0, 'memory_free': 0, 'gpu_utilization': 0, 'memory_utilization': 0, 'temperature': 0, 'timestamp': '2024-11-21T18:53:07.781830'}]
Recei

ConnectionClosedError: received 1012 (service restart); then sent 1012 (service restart)